In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, 
    roc_curve, roc_auc_score, recall_score, precision_score, 
    f1_score, auc, precision_recall_curve, make_scorer, 
    ConfusionMatrixDisplay
)

import imblearn
from imblearn.under_sampling import RandomUnderSampler

from scipy import stats
from scipy.stats import chisquare
from scipy.interpolate import interp1d

# Set seaborn style for plots
sns.set_style('darkgrid')

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, roc_curve, make_scorer
from sklearn.inspection import permutation_importance
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
# ייבוא הספריות הנדרשות
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_hist_gradient_boosting  # נדרש עבור HistGradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline

In [2]:
df = pd.read_csv("C:/Users/ronik/OneDrive - Ariel University/Final Project/Roni Project (2).csv")

In [3]:
df1 = df[0:7000]

In [4]:
df2 = df[7000:]

## Custom Transformer for Dropping Specified Columns in a DataFrame

In [5]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(columns=self.columns)

## Data Preprocessing Function for Feature Engineering and Cleaning

In [6]:
def prepare_data(df):
    df.columns = df.columns.str.lower()

    # Convert to datetime
    date_columns = ['first_deposit_date', 'last_deposit_date', 'account_create_on_date',
                    'first_closed_trade_date', 'last_closed_trade_date']
    
    for col in date_columns:
        df[col] = pd.to_datetime(df[col])

    # Create new columns
    df['days_between_deposits'] = (df['last_deposit_date'] - df['first_deposit_date']).dt.days
    df['days_to_first_deposit'] = (df['first_deposit_date'] - df['account_create_on_date']).dt.days
    df['days_between_trades'] = (df['last_closed_trade_date'] - df['first_closed_trade_date']).dt.days
    df['success_ratio_crads'] = df['num_crads_success'] / df['num_crads']
    df['approval_ratio_docs'] = df['docs_approved'] / df['docs_uploaded']
    df['avg_days_between_deposits'] = df['deposit_time_delta'] / df['deposit_attempts']
    df['avg_days_between_trades'] = df['signup_time_delta'] / df['num_closed_trades']
    df['first_deposit_month'] = df['first_deposit_date'].dt.month
    df['last_deposit_month'] = df['last_deposit_date'].dt.month
    df['first_deposit_weekday'] = df['first_deposit_date'].dt.weekday

    # Fill missing values
    fillna_columns = {
        'deposit_attempts': df['deposit_attempts'].mean(),
        'sucess_deposit_attempts': df['sucess_deposit_attempts'].mean(),
        'num_crads': df['num_crads'].mean(),
        'num_crads_success': df['num_crads_success'].mean(),
        'days_between_deposits': df['days_between_deposits'].mean(),
        'days_between_trades': df['days_between_trades'].mean(),
        'success_ratio_crads': 0,
        'approval_ratio_docs': df['approval_ratio_docs'].mean(),
        'avg_days_between_deposits': df['avg_days_between_deposits'].mean(),
        'last_deposit_month': df['last_deposit_month'].mode()[0]
    }

    for col, value in fillna_columns.items():
        df[col].fillna(value, inplace=True)

    # Handle division by zero and infinite values
    df['avg_days_between_trades'] = np.where(df['num_crads'] == 0, 0, df['signup_time_delta'] / df['num_crads'])
    df['avg_days_between_trades'].replace([np.inf, -np.inf], 0, inplace=True)
    
    # Additional step to handle infinite and large values
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.mean(), inplace=True)


    # Filter rows based on certain conditions
    df = df[df['calls'] <= 750]
    df = df[df['docs_uploaded'] < 100]
    df = df[df['days_to_first_deposit'] < 6039]

    return df


In [7]:
prepare_data(df2)

,user,deposit_attempts,sucess_deposit_attempts,num_crads,num_crads_success,first_deposit_date,last_deposit_date,deposit_sucess_ratio,deposit_time_delta,signup_time_delta,...,days_between_deposits,days_to_first_deposit,days_between_trades,success_ratio_crads,approval_ratio_docs,avg_days_between_deposits,avg_days_between_trades,first_deposit_month,last_deposit_month,first_deposit_weekday
7000,15581700,13,6,2,2,2024-01-10,2024-04-01,0.461538,82,127,...,82.0,127,107.0,1.0,1.000000,6.307692,63.5,1,4.0,2
7001,15776170,3,3,1,1,2023-11-20,2024-01-03,1.000000,44,11,...,44.0,11,62.0,1.0,0.500000,14.666667,11.0,11,1.0,0
7002,16383959,1,1,1,1,2024-05-30,2024-05-30,1.000000,0,0,...,0.0,0,7.0,1.0,0.735896,0.000000,0.0,5,5.0,3
7003,16376074,4,3,1,1,2024-05-30,2024-06-10,0.750000,11,3,...,11.0,3,55.0,1.0,0.400000,2.750000,3.0,5,6.0,3
7004,16361165,3,2,1,1,2024-05-21,2024-06-06,0.666667,16,0,...,16.0,0,65.0,1.0,0.222222,5.333333,0.0,5,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71986,16428364,2,1,1,1,2024-06-16,2024-06-16,0.500000,0,0,...,0.0,0,0.0,1.0,0.666667,0.000000,0.0,6,6.0,6
71987,16410162,1,1,1,1,2024-06-11,2024-06-11,1.000000,0,2,...,0.0,2,45.0,1.0,0.285714,0.000000,2.0,6,6.0,1
71988,16021854,1,1,1,1,2024-01-28,2024-01-28,1.000000,0,1,...,0.0,1,122.0,1.0,0.500000,0.000000,1.0,1,1.0,6
71989,15882594,0,0,0,0,2024-01-23,2024-01-23,0.000000,0,40,...,0.0,40,36.0,0.0,1.000000,10.589192,0.0,1,1.0,1


## Split Train and Test

In [10]:
cols = df2.columns.tolist()
target = 'is_ch_back'
cols = [c for c in cols if c not in [target]]

X = df2[cols]
Y = df2[target]

# StratifiedShuffleSplit for train-test split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1)

for train_index, test_index in sss.split(X, Y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

## Test Pipeline for Data Preprocessing and Classification with Undersampling

In [8]:
# Define numerical and categorical columns
date_columns = ['first_deposit_date', 'last_deposit_date', 'account_create_on_date',
                'first_closed_trade_date', 'last_closed_trade_date']

additional_columns = ['user', 'conversion_owner', 'tag']

numeric_columns = ['days_between_deposits', 'days_to_first_deposit', 'days_between_trades', 
                   'success_ratio_crads', 'approval_ratio_docs', 'avg_days_between_deposits', 
                   'avg_days_between_trades', 'first_deposit_month', 'last_deposit_month', 
                   'first_deposit_weekday']

categorical_columns = ['country_name', 'channel', 'ava_business', 'profile_state', 'business_group_name']

# Create the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('scaler', StandardScaler())
        ]), numeric_columns),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
        ]), categorical_columns)
    ],
    remainder='passthrough'
)

# Training pipeline with undersampling
training_pipeline = Pipeline(steps=[
    ('drop_columns', DropColumns(columns=date_columns + additional_columns)),
    ('preprocessor', preprocessor),
    ('undersample', RandomUnderSampler(sampling_strategy=0.5, random_state=42)),
    ('model', HistGradientBoostingClassifier(learning_rate=0.1, max_depth=3, max_iter=100, random_state=120))
])

# Train the pipeline
training_pipeline.fit(X_train, y_train)

Pipeline(steps=[('drop_columns',
                 DropColumns(columns=['first_deposit_date', 'last_deposit_date',
                                      'account_create_on_date',
                                      'first_closed_trade_date',
                                      'last_closed_trade_date', 'user',
                                      'conversion_owner', 'tag'])),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['days_between_depos...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['country_name', 'channel',
                                                   'ava_business',
                                                   'profile_state',
                                                   'business_group_name'])])),
                ('undersample',
                 RandomUnderSampler(random_state=42, sampling_strategy=0.5)),
                ('model',
                 HistGradientBoostingClassifier(max_depth=3,
                                                random_state=120))])

## Evaluation Metrics for Test Pipeline Prediction

In [10]:
# Make predictions
y_probabilities = training_pipeline.predict_proba(X_test)[:, 1]
y_pred = (y_probabilities >= 0.5).astype(int)

# Calculate evaluation metrics
cm = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred)
specificity = recall_score(y_test, y_pred, pos_label=0)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_probabilities)
fpr, tpr, _ = roc_curve(y_test, y_probabilities)

# Print evaluation metrics
print(f"Confusion Matrix:\n{cm}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Specificity: {specificity}")
print(f"F1-score: {f1}")
print(f"Accuracy: {accuracy}")
print(f"AUC: {auc}")


Confusion Matrix:
[[12814   129]
 [    4    52]]
Precision: 0.287292817679558
Recall: 0.9285714285714286
Specificity: 0.9900332225913622
F1-score: 0.4388185654008439
Accuracy: 0.9897684437264405
AUC: 0.9885252370282889


## Inference Pipeline for Predicting on New Data Using Trained Model

In [11]:
# Extract the trained model and preprocessor
trained_model = training_pipeline.named_steps['model']
trained_preprocessor = training_pipeline.named_steps['preprocessor']

# Create the inference pipeline without undersampling
inference_pipeline = Pipeline(steps=[
    ('drop_columns', DropColumns(columns=date_columns + additional_columns)),
    ('preprocessor', trained_preprocessor),
    ('model', trained_model)
])

# Use this pipeline to make predictions on new data
predictions = inference_pipeline.predict(X_test)

## Preprocessing and Predicting Fraud Probabilities on New Data Function

In [12]:
def preprocess_and_predict(new_data, inference_pipeline):
    # Prepare new data
    new_data = prepare_data(new_data)
    
    # Make predictions
    fraud_probabilities = inference_pipeline.predict_proba(new_data)[:, 1]
    
    # Add predictions to the original data
    final_new_data = new_data.copy()
    final_new_data['Fraud_Probability'] = fraud_probabilities
    
    
    # עיגול הערכים בעמודת Fraud_Probability לשלוש ספרות אחרי הנקודה
    final_new_data['Fraud_Probability'] = final_new_data['Fraud_Probability'].round(3)
    
    # הוספת עמודת זיהוי הונאה
    threshold = 0.5
    final_new_data['Fraud_Label'] = final_new_data['Fraud_Probability'] > threshold
    
    final_new_data['Fraud_Probability'] = final_new_data['Fraud_Probability']*100
    
    # מיון הנתונים לפי הסתברות להונאה בסדר יורד
    final_new_data = final_new_data.sort_values(by='Fraud_Probability', ascending=False)
    
    # בחירת העמודות להצגה
    selected_columns = ['user', 'conversion_owner', 'tag', 'country_name', 'business_group_name', 'account_create_on_date', 'Fraud_Probability', 'Fraud_Label']
    
    # הצגת הטבלה לאחר השינויים
    final_new_data = final_new_data[selected_columns]
    
    # שמירת הטבלה כקובץ CSV
    final_new_data[selected_columns].to_csv('fraud_predictions.csv', index=False)
    
    return final_new_data


## Example usage

In [13]:
#new_data = pd.read_csv('C:/Users/ronik/OneDrive - Ariel University/Final Project/All_Data.csv', encoding='latin1')  # Load new data
final_new_data = preprocess_and_predict(df1, inference_pipeline)
final_new_data

,user,conversion_owner,tag,country_name,business_group_name,account_create_on_date,Fraud_Probability,Fraud_Label
6683,15338894,AD Unassigned,avafx,Kuwait,Ava Financial,2023-07-26,100.0,True
1172,15140226,AvaFX Customer Service,158155,Tanzania,Ava Financial,2023-07-06,100.0,True
5846,15305791,AD Unassigned,125322,United Arab Emirates,Ava Financial,2023-07-24,100.0,True
4093,16281525,AD Unassigned,86776,Ecuador,Ava Financial,2024-04-23,100.0,True
4089,16185273,AD Unassigned,86776,Ecuador,Ava Financial,2024-03-19,100.0,True
...,...,...,...,...,...,...,...,...
2640,16375306,Jacob Hannah,125322,United Arab Emirates,Ava Financial,2024-05-27,0.0,False
2639,16129587,AvaFX Customer Service,171358,China,Ava Financial,2024-03-02,0.0,False
2638,15583892,Georgi Tuparov,equation_ie_search_nonbrand,Ireland,Ava Capital,2023-09-06,0.0,False
2637,14581901,Konstantin Dobrov,equation_uk_search_nonbrand,United Kingdom,Ava Capital,2023-01-30,0.0,False
